# IIIF presentation manifest generation

This notebook generates a IIIF presentation manifest for each file in the input directory (iteration TBD).

In [ ]:
!pip install git+https://github.com/giacomomarchioro/pyIIIFpres
!pip install toml
!pip install lxml_html_clean

import glob
import toml
import json
import requests

# Declaring inputs

files = glob.glob("./input/*.toml")

# Iterate over toml files in the input

for file in files:
  with open(file, 'r') as f:
    try:
      toml_data = toml.load(f)
      json_data = json.dumps(toml_data)
      parsed_json = json.loads(json_data)
      #print(f"File: {file}, Parsed JSON: {parsed_json}")

# Retrieve image dimensions and add to metadata

      mf_data_new = []

      # Iterate over the objects in mf_data (assuming it's a dictionary)
      if "mf_data" in toml_data:
        for mf_object_key, mf_object_value in toml_data["mf_data"].items():
          #print(f"File: {file}, mf_object_key: {mf_object_key}, mf_object_value: {mf_object_value}")

          # mf_data_new = []
          url = mf_object_value
          response = requests.get(url)
          if response.status_code == 200:
            mf_data = response.json()
            width = mf_data.get('width', 0)
            height = mf_data.get('height', 0)
            mf_data_new.append((mf_object_key, mf_object_value, str(width), str(height)))

          else:
            mf_data_new.append((mf_object_key, mf_object_value, 99999, 99999))

      else:
        print(f"File: {file} does not contain 'mf_data' key.")

      data = mf_data_new
      #print(data)

# Create IIIF presentation manifest

      from IIIFpres import iiifpapi3
      from IIIFpres.utilities import read_API3_json
      import urllib.parse
      iiifpapi3.BASE_URL = "https://iiif.annemarie-schwarzenbach.ch/"
      manifest = iiifpapi3.Manifest()
      manifest.set_id(extendbase_url=toml_data["mf_id"])
      manifest.add_label(toml_data["mf_lang"],toml_data["mf_label"])
      manifest.add_behavior("paged")

      for idx,d in enumerate(data):
          idx+=1
          canvas = manifest.add_canvas_to_items()
          canvas.set_id(extendbase_url="canvas/p%s"%idx) # in this case we use the base url
          canvas.set_height(d[3])
          canvas.set_width(d[2])
          canvas.add_label("en",d[0])
          annopage = canvas.add_annotationpage_to_items()
          annopage.set_id(extendbase_url="page/p%s/1" %idx)
          annotation = annopage.add_annotation_to_items(target=canvas.id)
          annotation.set_id(extendbase_url="annotation/p%s-image"%str(idx).zfill(4))
          annotation.set_motivation("painting")
          url = "".join(urllib.parse.quote(str(d[1]),safe='/')).replace('%3A', ':', 1)
          annotation.body.set_id("".join(url))
          annotation.body.set_type("Image")
          annotation.body.set_format("image/jpeg")
          annotation.body.set_width(d[2])
          annotation.body.set_height(d[3])
          s = annotation.body.add_service()
          s.set_id(url)
          s.set_type("ImageService3")
          s.set_profile("level1")

# Churn out the newly created IIIF manifest

      manifest.json_save(toml_data["mf_id"].join(".json"))
      print(manifest.json_dumps())

    except toml.TomlDecodeError as e:
      print(f"Error decoding TOML in file {file}: {e}")